# Seldon deployment of Alibi Outlier detector

Tne objective of this tutorial is to build a "loan approval" classifier equiped with the outliers detector from [alibi-detect](https://github.com/SeldonIO/alibi-detect) package.
The diagram of this tutorial is as follows:

**NOTE**: I will need to make new diagram for this variation of the tutorial

In this tutorial we will follow the following steps:

1) Train and test model to predict loan approvals

2) Train and test outliers detector

3) Containerise and deploy your models

4) Test your your new seldon deployment


## Before you start
Make sure you install the following dependencies, as they are critical for this example to work:

* Helm v3.0.0+
* A Kubernetes cluster running v1.13 or above (minkube / docker-for-windows work well if enough RAM)
* kubectl v1.14+
* ksonnet v0.13.1+
* kfctl 0.5.1 - Please use this exact version as there are major changes every few months
* Python 3.6+
* Python DEV requirements (we'll install them below)

You can follow this [notebook](../../../notebooks/seldon_core_setup.ipynb) to setup your cluster.

Let's get started! 🚀🔥 


## Install Python dependencies

In [ ]:
!pip install -r requirements-dev.txt

# Train and test loanclassifier

In [ ]:
!pygmentize train_classifier.py

In [ ]:
!python3 train_classifier.py

In [ ]:
!pygmentize pipeline/loanclassifier/Model.py

In [ ]:
import sys; sys.path.append("pipeline/loanclassifier")
from Model import Model

model = Model()

In [ ]:
import xai
import numpy as np

from train_classifier import load_data

data, X_train, y_train, X_test, y_test = load_data()
proba = model.predict(X_test)

pred = np.argmax(proba, axis=1)
xai.metrics_plot(y_test, pred)

# Train and test outliers detector

In [ ]:
!pygmentize train_detector.py

In [ ]:
!python3 train_detector.py

In [ ]:
!pygmentize pipeline/outliersdetector/Detector.py

In [ ]:
import sys; sys.path.append("pipeline/outliersdetector")
from Detector import Detector

detector = Detector()

In [ ]:
import json

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import confusion_matrix, f1_score
from alibi_detect.utils.data import create_outlier_batch


np.random.seed(1)
outlier_batch = create_outlier_batch(data.data, data.target, n_samples=1000, perc_outlier=10)
X_outlier, y_outlier = outlier_batch.data.astype('float'), outlier_batch.target

In [ ]:
od_preds = json.loads(detector.predict(X_outlier))

In [ ]:
labels = outlier_batch.target_names
y_pred = od_preds['data']['is_outlier']
f1 = f1_score(y_outlier, y_pred)
print('F1 score: {}'.format(f1))
cm = confusion_matrix(y_outlier, y_pred)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, cbar=True, linewidths=.5)
plt.show()

# Build and deploy components

### Build images

First, build s2i images using provided Makefile

In [ ]:
!pygmentize Makefile

In [ ]:
!make

or if using Minikube

In [ ]:
!eval $(minikube docker-env) && make

### Deploy separate loanclassifier

In [ ]:
!pygmentize pipeline/loanclassifier.yaml

In [ ]:
!kubectl apply -f pipeline/loanclassifier.yaml

### Deploy separate outliers detector

In [ ]:
!pygmentize pipeline/outliersdetector.yaml

In [ ]:
!kubectl apply -f pipeline/outliersdetector.yaml

### View newly deployed Kubernetes pods

In [ ]:
!kubectl get pods

# Test deployed components

In [ ]:
from seldon_core.seldon_client import SeldonClient

In [ ]:
to_explain = X_test[:1]
print(to_explain)

In [ ]:
sc = SeldonClient(
    gateway="ambassador", 
    deployment_name="loanclassifier",
    gateway_endpoint="localhost:8003",
    payload_type="ndarray",
    namespace="seldon",
    transport="rest"
)

prediction = sc.predict(data=to_explain)
prediction.response

In [ ]:
sc = SeldonClient(
    gateway="ambassador", 
    deployment_name="outliersdetector",
    gateway_endpoint="localhost:8003",
    payload_type="ndarray",
    namespace="seldon",
    transport="rest"
)

prediction = sc.predict(data=to_explain)
json.loads(prediction.response.strData)